In [ ]:
# import os
# import zipfile

# def unzip_folder(folder_path):
#     # Check if the folder exists
#     if not os.path.exists(folder_path):
#         print(f"Error: Folder '{folder_path}' not found.")
#         return

#     # Get a list of all files in the folder
#     files = os.listdir(folder_path)

#     # Iterate through each file in the folder
#     for file in files:
#         file_path = os.path.join(folder_path, file)

#         # Check if the file is a zip file
#         if file.endswith('.zip'):
#             print(f"Unzipping '{file}'...")

#             # Open the zip file
#             with zipfile.ZipFile(file_path, 'r') as zip_ref:
#                 # Extract all contents to the same folder
#                 zip_ref.extractall(folder_path)

#             print(f"Unzip complete for '{file}'.")

# # Example usage
# folder_path = "AudioGMM"
# unzip_folder(folder_path)


In [ ]:
# !pip install librosa
# !pip install tensorflow
# !pip install matplotlib

In [ ]:
class_mapping_list = ['right', 'eight', 'cat', 'tree', 'bed', 'happy', 'go', 'dog', 'no', 'wow',
                      'nine', 'left', 'stop', 'three', 'sheila', 'one', 'bird', 'zero', 'seven', 'up',
                      'marvin', 'two', 'house', 'down', 'six', 'yes', 'on', 'five', 'off', 'four']

# Creating a dictionary from the list
class_mapping_dict = {class_name: index for index, class_name in enumerate(class_mapping_list)}

# Example usage
print(class_mapping_dict)

{'right': 0, 'eight': 1, 'cat': 2, 'tree': 3, 'bed': 4, 'happy': 5, 'go': 6, 'dog': 7, 'no': 8, 'wow': 9, 'nine': 10, 'left': 11, 'stop': 12, 'three': 13, 'sheila': 14, 'one': 15, 'bird': 16, 'zero': 17, 'seven': 18, 'up': 19, 'marvin': 20, 'two': 21, 'house': 22, 'down': 23, 'six': 24, 'yes': 25, 'on': 26, 'five': 27, 'off': 28, 'four': 29}


In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
import librosa.display

def generate_mfcc_features(file_path, n_mfcc=20):
    # Load the audio file using librosa
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20)

    transposed_mfcc =  mfccs.T
    d_mfccs = np.gradient(transposed_mfcc, axis= 0)
    dd_mfccs = np.gradient(d_mfccs, axis= 0)
    # print(transposed_mfcc.shape)
    # print(delta_mfccs.shape)

    scaler = MinMaxScaler()

    scaled_mfccs = scaler.fit_transform(mfccs.T)
    scaled_dmfccs =  scaler.fit_transform(d_mfccs)
    scaled_ddmfccs = scaler.fit_transform(dd_mfccs)

    feature_vector = np.concatenate((scaled_mfccs,scaled_dmfccs), axis=1)
    feature_vector = np.concatenate((feature_vector,scaled_ddmfccs), axis=1)
    # print(feature_vector.shape)
    return feature_vector

In [ ]:
# Example usage
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

mfcc_dict = {}
model_dict = {}
pca_dict = {}

for i in class_mapping_list:
    directory_path = "AudioGMM/" + i
    files = os.listdir(directory_path)

    mfcc_list = []

    for file in files:
        file_path = os.path.join(directory_path, file)
        mfcc_list.append(generate_mfcc_features(file_path, 20))

    f_array = np.vstack(mfcc_list)
    print(f_array.shape)
    # sc = StandardScaler()
    # f_new = sc.fit_transform(f_array)
    f_new = f_array
    gmm = GaussianMixture(n_components=10, covariance_type='full')
    gmm.fit(f_new)
    model_dict[i] = gmm
    # pca_dict[i] = sc
    print(i)

(66614, 60)
right
(65960, 60)
eight
(49174, 60)
cat
(48368, 60)
tree
(48223, 60)
bed
(49195, 60)
happy
(66509, 60)
go
(49185, 60)
dog
(66624, 60)
no
(49628, 60)
wow
(66559, 60)
nine
(65957, 60)
left
(67465, 60)


/home/iiitd/.local/lib/python3.10/site-packages/sklearn/mixture/_base.py:268: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


stop
(65875, 60)
three
(48920, 60)
sheila
(66622, 60)
one
(49374, 60)
bird
(67346, 60)
zero
(68746, 60)
seven
(65781, 60)
up
(49906, 60)
marvin
(67084, 60)
two
(50339, 60)
house
(66810, 60)
down
(67267, 60)
six
(66841, 60)
yes
(66629, 60)
on
(65949, 60)
five
(66119, 60)
off
(66811, 60)
four


In [ ]:
def predictor(X):

    model_name = ""
    model_score = -1000000
    for i in class_mapping_list:
        # tempx = pca_dict[i].transform(X)
        temp = model_dict[i].score(X)
        if(temp>model_score):
            model_score = temp
            model_name = i
    return class_mapping_dict[model_name]

In [ ]:
import pandas as pd;

data = pd.read_csv("test.csv")
result_list = []
for i in data.values:
    tid = i[0]
    file_path = "SpeechCommandTest/" + i[1]
    X = generate_mfcc_features(file_path)
    predicted_class = predictor(X)
    result_list.append([tid,predicted_class])

In [ ]:
print(result_list)

[[0, 29], [1, 26], [2, 8], [3, 7], [4, 28], [5, 17], [6, 19], [7, 23], [8, 28], [9, 19], [10, 26], [11, 10], [12, 28], [13, 15], [14, 17], [15, 27], [16, 12], [17, 20], [18, 8], [19, 12], [20, 19], [21, 6], [22, 3], [23, 21], [24, 0], [25, 27], [26, 13], [27, 2], [28, 1], [29, 2], [30, 29], [31, 4], [32, 14], [33, 1], [34, 10], [35, 17], [36, 2], [37, 24], [38, 24], [39, 28], [40, 3], [41, 21], [42, 23], [43, 22], [44, 1], [45, 19], [46, 11], [47, 28], [48, 24], [49, 15], [50, 6], [51, 29], [52, 10], [53, 19], [54, 21], [55, 4], [56, 6], [57, 25], [58, 12], [59, 0], [60, 8], [61, 12], [62, 26], [63, 13], [64, 15], [65, 20], [66, 22], [67, 25], [68, 19], [69, 18], [70, 6], [71, 4], [72, 0], [73, 28], [74, 1], [75, 17], [76, 14], [77, 9], [78, 11], [79, 4], [80, 4], [81, 23], [82, 25], [83, 22], [84, 18], [85, 17], [86, 28], [87, 2], [88, 16], [89, 14], [90, 19], [91, 22], [92, 11], [93, 22], [94, 19], [95, 10], [96, 13], [97, 8], [98, 27], [99, 15], [100, 4], [101, 13], [102, 23], [103,

In [ ]:
df = pd.DataFrame(result_list, columns=["ID", "TARGET"])

# Specify the CSV file path
csv_file_path = "output.csv"

# Write the DataFrame to the CSV file
df.to_csv(csv_file_path, index=False)